
---

# 🛠️ Custom Serialization

### 🎯 Intent

* Decide how your model outputs data when converting to dict/JSON.
* This helps with masking secrets, formatting values, reshaping payloads, and keeping API responses clean.
---

### 🧩 Core Components

1. **🧪 `@field_serializer`**

   * Customize output of a **single field**.
   * Example: mask passwords, format decimals, pretty phone numbers.

2. **🏗️ `@model_serializer`**

   * Override the **entire model’s output**.
   * Useful for reshaping payloads (internal vs public API).

3. **🔁 Serializer Modes**

   * `plain` → transform the raw value directly.
   * `wrap` → get `(value, serializer)` so you can extend defaults.

4. **📦 Nested & Collections**

   * Works inside lists/dicts/sets of models.
   * Combine with `include` / `exclude` for fine control.

5. **🧭 Aliases + Serializers**

   * Works with `Field(alias="...")` and `by_alias=True`.
   * Remap fields to external names if needed.

6. **⚙️ Dump Options**

   * Serializers still honor:

     * `exclude_none`
     * `exclude_unset`
     * `exclude_defaults`
     * `mode="python" | "json"`

7. **🛡️ Masking & Privacy**

   * Central place to hide sensitive data (passwords, tokens).
   * Raw values stay safe internally → only masked in dumps.

8. **⏱ Formatting Helpers**

   * Datetime → ISO or custom format.
   * Decimal/Money → fixed precision.
   * Enum → output `.value` or friendly label.

9. **💡 Tips**

   * Use **field serializers** for local tweaks.
   * Use **model serializer** for reshaping.
   * Prefer `wrap` mode to extend default behavior.

---


In [1]:
from pydantic import BaseModel, Field, SecretStr, field_serializer, model_serializer
from decimal import Decimal
from datetime import datetime

# 🎯 User model with custom serialization
class User(BaseModel):
    id: int
    email: str
    password: SecretStr
    balance: Decimal = Field(default=0)
    joined_at: datetime

    # 🧪 Field-level serializer → mask email domain
    @field_serializer("email")
    def mask_email(self, email: str) -> str:
        name, domain = email.split("@")
        return f"{name}@***"

    # 🧪 Field-level serializer → format balance
    @field_serializer("balance")
    def format_balance(self, value: Decimal) -> str:
        return f"${value:.2f}"

    # 🏗 Model-level serializer → reshape output
    @model_serializer(mode="wrap")
    def to_public(self, handler):
        data = handler(self)   # default dict
        # Remove password entirely from output
        data.pop("password", None)
        # Add a custom "summary" field
        data["summary"] = f"User {self.id} joined at {self.joined_at.date()}"
        return data


# ⚡ Example usage
if __name__ == "__main__":
    user = User(
        id=1,
        email="mukesh@example.com",
        password="supersecret",
        balance=Decimal("1234.5"),
        joined_at=datetime(2024, 1, 15, 10, 30),
    )

    # 📦 Dump as dict
    print(user.model_dump())
    # 📤 Dump as JSON string
    print(user.model_dump_json(indent=2))


{'id': 1, 'email': 'mukesh@***', 'balance': '$1234.50', 'joined_at': datetime.datetime(2024, 1, 15, 10, 30), 'summary': 'User 1 joined at 2024-01-15'}
{
  "id": 1,
  "email": "mukesh@***",
  "balance": "$1234.50",
  "joined_at": "2024-01-15T10:30:00",
  "summary": "User 1 joined at 2024-01-15"
}
